In [1]:
!pip install tensorflow==1.13.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.4 MB/s eta 0:00:0000:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.6/367.6 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.4
    Uninstalling tensorflow-2.6.4:
      Successfully uninstalled tensorflow-2.6.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [2]:
import tensorflow as tf
import os
import time
import pickle
import numpy as np

print(tf.__version__)
CIFAR_DIR = '/kaggle/input/cifar10/cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.1
['data_batch_1', 'data_batch_2', 'batches.meta', 'test_batch', 'data_batch_3', 'data_batch_5', 'data_batch_4', 'readme.html']


In [3]:
train_filenames = [os.path.join(CIFAR_DIR, f'data_batch_{i}') for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]
print(train_filenames)
print(test_filenames)

['/kaggle/input/cifar10/cifar-10-batches-py/data_batch_1', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_2', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_3', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_4', '/kaggle/input/cifar10/cifar-10-batches-py/data_batch_5']
['/kaggle/input/cifar10/cifar-10-batches-py/test_batch']


In [4]:
def load_data(filename):
    """前面是图片特征，后面是图片分类标签"""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']


# tensorflow.Dataset
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_lables = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)  # all_data[0]列表中10000个存有(32*32*3=)3072个数的一维列表c
            all_lables.append(labels)  # all_data[0]列表中10000个标签值
        self._data = np.vstack(all_data)  # 垂直方向沿轴2堆叠 变为ndarray
        self._data = self._data / 127.5 - 1  # 归一化到范围[-1,1]
        self._lables = np.hstack(all_lables)  # 一维列表变成ndarray
        print(self._data.shape)  # (50000, 3072) 每一张图像展平后的结果
        print(self._lables.shape)
        print(self._lables[:5])

        self._num_samples = self._data.shape[0]  # 样本数
        self._need_shuffle = need_shuffle  # 进行洗牌随机
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()

    def _shuffle_data(self):
        # 通过随机排列下标进行随机样本
        p = np.random.permutation(self._num_samples)  # 变成随机下标
        self._data = self._data[p]
        self._lables = self._lables[p]

    def next_batch(self, batch_size):
        """return batch_size samples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_samples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = self._indicator + batch_size
            else:
                raise Exception('have no more samples')
        if end_indicator > self._num_samples:
            # 去除不足batch_size剩余的样本
            raise Exception('batch size is larger than the remaining samples')
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._lables[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels


train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
[6 9 9 4 1]
(10000, 3072)
(10000,)
[3 8 8 0 6]


In [5]:
t = tf.constant(np.ones((100, 32, 32, 32)))
print(tf.pad(t, [[0, 0], [0, 0], [0, 0], [16, 16]]))

Tensor("Pad:0", shape=(100, 32, 32, 64), dtype=float64)


In [6]:
x = tf.constant(np.ones((10, 32, 32, 32)))
test = tf.layers.conv2d(x,
                        64,
                        (7, 7),
                        strides=(2, 2),
                        padding='same',
                        activation=tf.nn.relu,
                        name='conv_test')
print(test) # Tensor("conv_test/Relu:0", shape=(10, 16, 16, 64), dtype=float64)
print(type(test)) #< class 'tensorflow.python.framework.ops.Tensor'>
print(test.get_shape()) # (10, 16, 16, 64)

Tensor("conv_test/Relu:0", shape=(10, 16, 16, 64), dtype=float64)
<class 'tensorflow.python.framework.ops.Tensor'>
(10, 16, 16, 64)


In [7]:
def residual_block(x, output_channel):
    """某个残差块的实现"""
    input_channel = x.get_shape().as_list()[-1]
    # 如果输出通道是输入通道的两倍，需要增加维度increase_dim=True
    if input_channel * 2 == output_channel:
        increase_dim = True
        strides = (2, 2)
    # 如果输入通道和输出通道一致，图像大小保持不变
    elif input_channel == output_channel:
        increase_dim = False
        strides = (1, 1)
    else:
        raise Exception("input channel can't match output channel")
    conv1 = tf.layers.conv2d(x,
                             output_channel,
                             (3, 3),
                             strides=strides,
                             padding='same',
                             activation=tf.nn.relu,
                             name='conv1')
    conv2 = tf.layers.conv2d(conv1,
                             output_channel,
                             (3, 3),
                             strides=(1, 1),
                             padding='same',
                             activation=tf.nn.relu,
                             name='conv2')
    if increase_dim:
        # 平均值和最大值池化都可以选
        pooled_x = tf.layers.average_pooling2d(x,
                                               (2, 2),
                                               (2, 2),
                                               padding='valid')
        padded_x = tf.pad(pooled_x,
                          [[0, 0], [0, 0], [0, 0],
                           [input_channel // 2, input_channel // 2]])
    else:
        padded_x = x
    # 确保形状尺寸相同才能残差连接
    output_x = conv2 + padded_x
    return output_x


def ResNet(x, num_residual_blocks,  # 每一层的残差块数目
           num_filter_base,  # 通道数
           class_num):
    """residual network implementation"""
    # 降采样次数 残差块数目
    num_subsampling = len(num_residual_blocks)
    layers = []
    input_size = x.get_shape().as_list()[1:]
    with tf.variable_scope('conv0'):
        # 首先经过一个普通卷积
        conv0 = tf.layers.conv2d(x,
                                 num_filter_base,
                                 (3, 3),
                                 strides=(1, 1),
                                 padding='same',
                                 activation=tf.nn.relu,
                                 name='conv0')
        layers.append(conv0)
    for sample_id in range(num_subsampling):
        for i in range(num_residual_blocks[sample_id]):
            with tf.variable_scope(f'conv{sample_id}_{i}'):
                conv = residual_block(layers[-1],
                                      num_filter_base * (2 ** sample_id))  # 通道数不断翻倍
                layers.append(conv)
    # 判断最终的形状是否正确
    multipliter = 2 ** (num_subsampling - 1)
    assert layers[-1].get_shape().as_list()[1:]\
            == [input_size[0] / multipliter,
                input_size[1] / multipliter,
                num_filter_base * multipliter]
    with tf.variable_scope('fc'):
        global_pool = tf.reduce_mean(layers[-1], [1, 2])  # 对width(1轴)和height(2轴)求平均
        logits = tf.layers.dense(global_pool, class_num)
        layers.append(logits)
    return layers[-1]

In [8]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
x_image = tf.reshape(x, [-1, 3, 32, 32])
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])
y_ = ResNet(x_image, [2, 3, 2], 32, 10)
print(y_.get_shape())
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)  # y是真实值
# 计算张量维度上元素的平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
with tf.name_scope('train_op'):
    # 这里计算梯度并更新了梯度
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

(?, 10)


In [9]:
init = tf.global_variables_initializer()  # 低版本tf必须要做
batch_size = 20
train_steps = 10000
test_steps = 100
with tf.Session() as s:
    s.run(init)  # 低版本tf必须要做
    start = time.time()
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, accu_val, _ = s.run([loss, accuracy, train_op],
                                      feed_dict={x: batch_data, y: batch_labels})
        if (i + 1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, accuracy: %4.5f'
                  % (i + 1, loss_val, accu_val))
            end = time.time()
            print(f'just 100 steps take time: {end - start}s')
            start=end
        if (i + 1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)  # 重新拿测试集
            all_test_accu_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_accu_val = s.run([accuracy],
                                      feed_dict={x: test_batch_data, y: test_batch_labels})
                all_test_accu_val.append(test_accu_val)
            test_accu = np.mean(all_test_accu_val)
            print('[Test ] Step: %d, accuracy: %4.5f' % (i + 1, test_accu))

2022-07-29 01:47:27.745103: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2022-07-29 01:47:27.749267: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000170000 Hz
2022-07-29 01:47:27.749629: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55bd179654f0 executing computations on platform Host. Devices:
2022-07-29 01:47:27.749661: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>


[Train] Step: 100, loss: 1.89894, accuracy: 0.15000
just 100 steps take time:51.49335980415344s
[Train] Step: 200, loss: 1.83713, accuracy: 0.45000
just 100 steps take time:51.30489730834961s
[Train] Step: 300, loss: 1.99875, accuracy: 0.25000
just 100 steps take time:50.69451022148132s
[Train] Step: 400, loss: 1.62546, accuracy: 0.40000
just 100 steps take time:50.85912036895752s
[Train] Step: 500, loss: 1.38188, accuracy: 0.60000
just 100 steps take time:50.74187183380127s
[Train] Step: 600, loss: 1.41893, accuracy: 0.30000
just 100 steps take time:51.5678608417511s
[Train] Step: 700, loss: 1.65330, accuracy: 0.40000
just 100 steps take time:50.965763568878174s
[Train] Step: 800, loss: 1.95867, accuracy: 0.20000
just 100 steps take time:50.82271432876587s
[Train] Step: 900, loss: 1.40928, accuracy: 0.50000
just 100 steps take time:51.45769000053406s
[Train] Step: 1000, loss: 1.27976, accuracy: 0.55000
just 100 steps take time:50.44248175621033s
(10000, 3072)
(10000,)
[3 8 8 0 6]
now 